# Group Relative Policy Optimization (GRPO) - Explanation and Implementation


### Policy in Reinforcement Learning?

A policy ($\pi$) is a strategy that an agent follows to decide what action to take in a given state.
The policy represents a probability distribution over possible outputs for a given input question ($\text{q}$).

### Policy Optimization

Policy optimization techniques are used to find the best policy that maximizes the expected return.

The goal of GRPO is to optimize the policy $\pi_\theta$ by comparing it with the old policy $\pi_{\theta_\text{old}}$ and updating it based on group-wise feedback.



 Let's now break down the GRPO algorithm step by step:

*Scenario: Assume we have an AI system that generates responses to user questions.*
- The system generates multiple possible answers (outputs) for each question.
- We use a group of these responses to estimate which ones are relatively better.
- Rewards are assigned based on user feedback or an evaluation metric.
- Higher rewards are assigned to correct answers, and lower rewards to incorrect answers.

*Example*

**Question**: "What is the capital of France?"

Possible outputs generated by the AI system {from $\pi_{\theta_\text{old}}$}: ["Paris", "London", "Berlin", "Madrid"]

in this case, the correct answer is "Paris".
- "Paris" (correct, high reward)
- "London" (incorrect, low reward)
- "Berlin" (incorrect, low reward)
- "Madrid" (incorrect, low reward)

therefore, the rewards assigned to the outputs can be: [0.9, 0.1, 0.1, 0.1]
<!-- Assigned rewards: {0.9, 0.1, 0.1, 0.1} -->


In [1]:
import numpy as np

# Define the function that samples from the old policy outputs
def sample_from_old_policy(old_policy_outputs, probabilities, sample_size=3):
    """
    Function that samples from the old policy outputs, given the probabilities of each output
    where:
    old_policy_outputs: list of strings, the outputs of the old policy
    probabilities: list of floats, the probability of each output
    sample_size: int, the number of samples to draw.
    
    return the sampled outputs based on thier probabilities
    """
    return np.random.choice(old_policy_outputs, size=sample_size, p=probabilities, replace=False)

# Define old policy outputs and their probabilities
old_policy_outputs = ["Paris", "London", "Berlin", "Madrid"]
probabilities = [0.4, 0.2, 0.2, 0.2]  # Probability of selecting each output

# Sample outputs
sampled_outputs = sample_from_old_policy(old_policy_outputs, probabilities)
print("Old Policy Outputs:", old_policy_outputs)
print("Sampled Outputs:", sampled_outputs)

Old Policy Outputs: ['Paris', 'London', 'Berlin', 'Madrid']
Sampled Outputs: ['Berlin' 'Paris' 'London']


## Step 2: The Advantage Function (Ai)

An advantage function (${A_{i}}$) helps determine how much better or worse an output is relative to the others in the group. The advantage function is tyically calculated as the difference between the reward of the output and the average reward of the group.
Given as:
$$A_{i} = R_{i} - \text{mean}(R_G)$$

where:
- $A_{i}$: Advantage of output $i$
- $R_{i}$: Reward of output $i$
- $R_{G}$: Rewards of all outputs in the group = $\{r_{1}, r_{2}, ..., r_{G}\}$
- $\text{mean}(R_G)$: Mean reward of the group

In our example, the advantage function for each output would be:


$$\text{mean}(R_G) = \frac{0.9 + 0.1 + 0.1 + 0.1}{4} = 0.3$$

the advantage values would be:
- "Paris": $0.9 - 0.3 = 0.6$
- "London": $0.1 - 0.3 = -0.2$
- "Berlin": $0.1 - 0.3 = -0.2$
- "Madrid": $0.1 - 0.3 = -0.2$

Advantage values: [0.6, -0.2, -0.2, -0.2]



In [2]:
def compute_advantages_raw(rewards):
    """
    Function that computes the advantages given the rewards
    where:
    rewards: list of floats, the rewards for each time step
    
    return the advantages for each time step
    """
    # Compute the advantages
    mean_reward = np.mean(rewards)
    advantages = np.array(rewards) - mean_reward
    return advantages

# Define rewards
rewards = [0.9, 0.1, 0.1, 0.1]
advantages_raw = compute_advantages_raw(rewards)
print("Advantages (Raw):", advantages_raw)

Advantages (Raw): [ 0.6 -0.2 -0.2 -0.2]



To fairly compare the outputs, we use the advantage values instead of the raw rewards.

We normalize the advantage values by dividing them by their standard deviation to make the training process more stable.
> ie after normalizing the advantage values, they are expected to have a mean of 0 and a standard deviation of 1.

The advantage is now computed as:

\begin{equation}
( A_i = \frac{r_i - \text{mean}(\{r_1, r_2, \dots, r_G\})}{\text{std}(\{r_1, r_2, \dots, r_G\})} )
\end{equation}

*If the standard deviation is 0 (i.e., all rewards are the same), we set it to 1 to avoid division errors.*

This results in a centered and scaled advantage score for each output.


In [3]:
def compute_advantages(rewards):
    mean_reward = np.mean(rewards)
    std_reward = np.std(rewards) if np.std(rewards) > 0 else 1  # Avoid division by zero
    advantages = [(r - mean_reward) / std_reward for r in rewards]
    return advantages

# Define rewards corresponding to sampled outputs
rewards = [0.9, 0.1, 0.1, 0.1]  # Example rewards
advantages = compute_advantages(rewards)
print("Rewards:", rewards)
print("Advantages (Raw)", advantages_raw)
print("Advantages Normalized:", advantages)

Rewards: [0.9, 0.1, 0.1, 0.1]
Advantages (Raw) [ 0.6 -0.2 -0.2 -0.2]
Advantages Normalized: [1.732050807568877, -0.5773502691896257, -0.5773502691896257, -0.5773502691896257]


Interpretation of Advantages Output:

- The first output (Paris) has a positive advantage (1.732), meaning it's much better than the group average.
- The other outputs (London, Berlin, Madrid) have negative advantages (-0.577), indicating they are below average.
- This advantage value is used to update the policy by rewarding better responses and penalizing worse ones.


## Step 3: Importance Sampling Ratio

The importance sampling ratio helps measure how much the new policy differs from the old one.

This is calculated by comparing the probabilities assigned to the outputs by the old and new policies.

considering the output $o_i$ for question $q$, the importance sampling ratio is given as:

$$\text{ratio} = \frac{\pi_{\theta}(o_i | q)}{\pi_{\theta_{\text{old}}}(o_i | q)}$$

where:
- $\pi_{\theta}(o_i | q)$: Probability of output $o_i$ given question $q$ under the new policy $\pi_{\theta}$
- $\pi_{\theta_{\text{old}}}(o_i | q)$: Probability of output $o_i$ given question $q$ under the old policy $\pi_{\theta_{\text{old}}}$
- If ratio > 1, the new policy assigns a higher probability to this output than before.
- If ratio < 1, the new policy assigns a lower probability than before.
- This helps in adjusting the policy weights without making drastic changes.

> *policy weights* are parameters that determine the probability distribution of outputs for a given input question. These weights are updated during training to improve the policy's performance.


In [4]:
def compute_importance_sampling_ratios(new_policy_probs, old_policy_probs):
    return [new / old for new, old in zip(new_policy_probs, old_policy_probs)]

# Example probabilities under the new and old policies
new_policy_probs = [0.5, 0.2, 0.15, 0.15]  # Hypothetical new policy probabilities
old_policy_probs = [0.4, 0.2, 0.2, 0.2]  # Old policy probabilities

ratios = compute_importance_sampling_ratios(new_policy_probs, old_policy_probs)
print("Importance Sampling Ratios:", ratios)


Importance Sampling Ratios: [1.25, 1.0, 0.7499999999999999, 0.7499999999999999]


Interpretation of Importance Sampling Ratios Output:
- Importance Sampling Ratios: [1.25, 1.0, 0.75, 0.75]
- The first output has a ratio of 1.25, meaning the new policy gives it a 25% higher probability than the old policy.
- The second output remains unchanged (ratio = 1.0), meaning no change in probability.
- The last two outputs have a ratio of 0.75, meaning they are assigned 25% lower probability under the new policy.
- These adjustments help reinforce good responses and reduce the probability of bad responses.


## Step 4: Clipping to Ensure Stability

To prevent excessive policy updates, we **clip** the importance sampling ratio.
- Clipping helps ensure that updates remain stable and controlled.
- The formula for clipping is:

clipped_ratio = clip(ratio, 1 - ε, 1 + ε)

$$\text{clipped\_ratio} = \text{clip}(ratio, 1 - \epsilon, 1 + \epsilon)$$

where:
- $\epsilon$: A small constant (e.g., 0.1) to limit the ratio's range.
- If the ratio exceeds (1 + ε), it is capped at (1 + ε).
- Similarly, if the ratio falls below (1 - ε), it is raised to (1 - ε).
- This avoids drastic changes in policy probability, ensuring smooth learning.

<!-- We clip the ratio to avoid making drastic updates: -->
<!-- 
clipped_ratio = clip(ratio, 1 - ε, 1 + ε)


where:

- Why? If we allow unbounded updates, training becomes unstable.
- Clipping ensures that probability changes remain within a safe range, preventing excessive updates that could cause instability. -->


In [5]:
def clip_ratios(ratios, epsilon=0.2):
    return [max(1 - epsilon, min(r, 1 + epsilon)) for r in ratios]

# Clipping importance sampling ratios
clipped_ratios = clip_ratios(ratios)
print("Clipped Ratios:", clipped_ratios)

Clipped Ratios: [1.2, 1.0, 0.8, 0.8]


Interpretation of Clipped Ratios Output:
- Clipping ensures that probability adjustments remain within the range [0.8, 1.2].
- This prevents overly aggressive updates, keeping training stable.

# Step 5: Policy Objective Function

The policy objective function is then used to update the policy weights through gradient ascent. This function combines the clipped surrogate objective and the advantage function to update the policy weights.

The policy objective function ensures that policy updates lead to improved performance.

The clipped surrogate objective is defined as:

$$L_{\text{clip}}(\theta) = \text{min}(ratio \cdot A, \text{clipped\_ratio} \cdot A)$$

where:
- $L_{\text{clip}}(\theta)$: Clipped surrogate objective function for policy update
- $ratio$: Importance sampling ratio
- $A$: Advantage function
- $\text{clipped\_ratio}$: Clipped importance sampling ratio
- The minimum of the two terms is taken to balance policy updates.
  - If the ratio leads to a large update, the clipped ratio is used instead to stabilize learning.
  - If the ratio leads to a small update, the original ratio is used.
- This prevents overly aggressive policy updates.


In [6]:
def compute_policy_objective(clipped_ratios, advantages):
    objective_values = [min(r * A, r * A) for r, A in zip(clipped_ratios, advantages)]
    return objective_values

# Compute policy objective
policy_objective = compute_policy_objective(clipped_ratios, advantages)
print("Policy Objective Values:", policy_objective)

Policy Objective Values: [2.078460969082652, -0.5773502691896257, -0.4618802153517006, -0.4618802153517006]


Interpretation of Policy Objective Values:
- Positive values indicate an increase in probability for that action.
- Negative values suggest reducing the probability of selecting that action.
- The clipping ensures that no single update is too large, preventing instability.

> instability is defined by poor convergence or oscillations in the training process, which can hinder learning.

## Step 6: KL Divergence Regularization

Divergence is a measure of how much one probability distribution differs from another.
Divergence is used to penalize large deviations between the new policy and the old policy.

Divergence regularization helps stabilize training by limiting policy updates. This can be done by adding a divergence penalty to the policy objective function.

KL divergence measures the difference between the new policy and the old policy.
Kullback-Leibler(KL) divergence, a measure of how one probability distribution diverges from a second, expected probability distribution denoted as ${D_{\text{KL}}(P\parallel Q)}$.

The standard formula for KL divergence is:

$$D_{\text{KL}}(P\parallel Q)=\sum _{x\in {\mathcal {X}}}P(x)\ \log \left({\frac {\ P(x)\ }{Q(x)}}\right).$$

where:
- $P(x)$: Probability of event $x$ under the new policy
- $Q(x)$: Probability of event $x$ under the old policy 
- The sum is taken over all possible events $x$ in the sample space $\mathcal{X}$.
- A high KL divergence suggests a large difference between the policies, requiring regularization.

> also referred to as relative entropy, information divergence, information gain and I divergence.


In [7]:
def compute_kl_divergence(old_policy_probs, new_policy_probs):
    kl_div = sum(o * np.log(o / n) for o, n in zip(old_policy_probs, new_policy_probs) if o > 0 and n > 0)
    return kl_div

kl_divergence = compute_kl_divergence(old_policy_probs, new_policy_probs)
print("KL Divergence:", kl_divergence)

KL Divergence: 0.025815408455028527




the KL divergence in GRPO is given as:

$$
D_{KL}(\pi_\theta \mid\mid \pi_{ref}) = \frac{\pi_\theta(o_i \mid q)}{\pi_{ref}(o_i \mid q)} - \log \frac{\pi_\theta(o_i \mid q)}{\pi_{ref}(o_i \mid q)} - 1
$$

where:
- $\pi_\theta(o_i \mid q)$: Probability of output $o_i$ given question $q$ under the new policy $\pi_{\theta}$
- $\pi_{ref}(o_i \mid q)$: Probability of output $o_i$ given question $q$ under the reference policy $\pi_{ref}$
- The KL divergence is calculated for each output and summed over all outputs.
- The regularization term penalizes large KL divergence, ensuring policy updates are controlled.

> reference policy $\pi_{ref}$ is either the old policy $\pi_{\theta_{\text{old}}}$ or a fixed policy used for stability.


In [10]:
def compute_kl_penalty(old_policy_probs, ref_policy_probs):
    kl_penalty = sum((new / ref) - np.log(new / ref) - 1 for new, ref in zip(old_policy_probs, ref_policy_probs) if new > 0 and ref > 0)
    return kl_penalty


def compute_grpo_kl_divergence(old_policy_probs, ref_policy_probs):
    kl_div= sum((new / ref) - np.log(new / ref) - 1 for new, ref in zip(old_policy_probs, ref_policy_probs) if new > 0 and ref > 0)
    return kl_div

# Reference policy probabilities (could be same as old policy or a smoothed version)

kl_grpo_divergence = compute_grpo_kl_divergence(old_policy_probs, new_policy_probs)
print("GRPO KL Divergence:", kl_grpo_divergence)



KL Penalty (Reference): 0.17040418085046793
KL Divergence (Old vs New): 0.11444607307731469
GRPO KL Divergence: 0.10222059358935232


Interpretation of KL Divergence:
- KL Divergence = 0.025815408455028527 means the new policy is slightly different from the old one.
- A low value suggests the policy update is stable and not drastically changing behavior.
- A high KL divergence would indicate that the new policy is deviating significantly, which could lead to instability.
- Regularization helps by penalizing large deviations, ensuring smooth learning progress.

# Step 7: Final GRPO Objective
 - The final objective function combines the clipped loss and KL divergence penalty:
   J_GRPO(θ) = mean(L) - β * D_KL

 Now, let's implement each of these steps in Python, one by one.


$$
J_{GRPO}(\theta) = E\left[ \sum_{i=1}^G \left( \min\left( \frac{\pi_{\theta_{old}}(o_i \mid q)}{\pi_\theta(o_i \mid q)} A_i, \text{clip}\left( \frac{\pi_{\theta_{old}}(o_i \mid q)}{\pi_\theta(o_i \mid q)}, 1-\varepsilon, 1+\varepsilon \right) A_i \right) - \beta D_{KL}(\pi_\theta \mid\mid \pi_{ref}) \right) \right]
$$